# Final Model Evaluation

This notebook evaluates the performance of our best food recognition model across training, validation, and test datasets. We'll analyze various metrics and visualize sample predictions to assess model quality.

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
import cv2
from pathlib import Path
import yaml
import random
from ultralytics import YOLO
from IPython.display import display, Image as IPImage

# Set plotting style
plt.style.use('ggplot')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = [14, 8]

# Fix seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

## 1. Load Best Model and Configuration

First, we'll load our best model and the training configuration. According to the training results, the best model is from epoch 40 (final epoch) which achieved the highest mAP scores.

In [ ]:
import os
from pathlib import Path

# Set paths
project_root = '..'
runs_dir = Path(os.path.join(project_root, 'runs/food_seg_model/food_recognition2'))
data_dir = Path(os.path.join(project_root, 'datasets/yolo_food_dataset'))

# Load training results
results_path = os.path.join(runs_dir, 'results.csv')
results_df = pd.read_csv(results_path)

# Display the results from the last few epochs
results_df.tail()

In [ ]:
# Find the best model based on mAP50-95(M) - mean Average Precision for masks
best_epoch_idx = results_df['metrics/mAP50-95(M)'].idxmax()
best_epoch = results_df.loc[best_epoch_idx, 'epoch']
best_map = results_df.loc[best_epoch_idx, 'metrics/mAP50-95(M)']

print(f"Best model found at epoch {best_epoch} with mAP50-95(M) = {best_map:.5f}")

# Load the best model
model_path = os.path.join(runs_dir, 'weights', 'best.pt')
if not os.path.exists(model_path):
    # Fallback to the last model if best.pt doesn't exist
    model_path = os.path.join(runs_dir, 'weights', 'last.pt')

model = YOLO(model_path)
print(f"Loaded model from {model_path}")

## 2. Dataset Preparation

Let's set up the datasets for evaluation. We need to ensure we have access to the training, validation, and test datasets.

In [ ]:
# Load dataset configuration
dataset_yaml = os.path.join(data_dir, 'dataset.yaml')
with open(dataset_yaml, 'r') as file:
    data_config = yaml.safe_load(file)

print("Dataset configuration:")
for key, value in data_config.items():
    print(f"  {key}: {value}")

# Define dataset paths
train_path = os.path.join(data_dir, 'train.yaml')
val_path = os.path.join(data_dir, 'val.yaml')

print(f"\nTrain path: {train_path}")
print(f"Validation path: {val_path}")

# Check if paths exist
for path, name in [(train_path, "Training"), (val_path, "Validation")]:
    if os.path.exists(path):
        if os.path.isdir(path):
            num_images = len([f for f in os.listdir(path) if f.endswith(('.jpg', '.png'))])
        else:
            num_images = "YAML file exists"
        print(f"{name} dataset: {num_images}")
    else:
        print(f"{name} dataset path does not exist: {path}")

## 3. Model Evaluation

Now we'll evaluate the model on each dataset to compare its performance. We'll use the YOLO model's built-in validation functionality to generate metrics.

In [ ]:
# Evaluate on training set
print("Evaluating on training set...")
train_metrics = model.val(data=train_path, verbose=True)

# Evaluate on validation set
print("\nEvaluating on validation set...")
val_metrics = model.val(data=val_path, verbose=True)

In [ ]:
# Extract key metrics for comparison
datasets = ['Training', 'Validation']
metrics_list = [train_metrics, val_metrics]

# Collect metrics into a dictionary
metrics_data = {
    'Dataset': datasets,
    'Precision (B)': [m.box.mp for m in metrics_list],
    'Recall (B)': [m.box.mr for m in metrics_list],
    'mAP50 (B)': [m.box.map50 for m in metrics_list],
    'mAP50-95 (B)': [m.box.map for m in metrics_list],
    'Precision (M)': [m.seg.mp for m in metrics_list],
    'Recall (M)': [m.seg.mr for m in metrics_list],
    'mAP50 (M)': [m.seg.map50 for m in metrics_list],
    'mAP50-95 (M)': [m.seg.map for m in metrics_list],
    'Inference Time (ms)': [m.speed['inference'] for m in metrics_list],
    # 'NMS Time (ms)': [m.speed['nms'] for m in metrics_list]
}

# Create a DataFrame for easy visualization
metrics_df = pd.DataFrame(metrics_data)
metrics_df

## 4. Metrics Visualization

Let's visualize the key metrics across datasets to better understand model performance.

In [ ]:
# Plot precision and recall
plt.figure(figsize=(16, 8))

# Bounding box precision/recall
plt.subplot(1, 2, 1)
bar_width = 0.35
index = np.arange(len(datasets))

plt.bar(index, metrics_data['Precision (B)'], bar_width, label='Precision (B)', color='steelblue')
plt.bar(index + bar_width, metrics_data['Recall (B)'], bar_width, label='Recall (B)', color='lightcoral')

plt.xlabel('Dataset')
plt.ylabel('Value')
plt.title('Bounding Box Precision and Recall')
plt.xticks(index + bar_width / 2, datasets)
plt.legend()
plt.ylim(0, 1.0)

# Mask precision/recall
plt.subplot(1, 2, 2)
plt.bar(index, metrics_data['Precision (M)'], bar_width, label='Precision (M)', color='steelblue')
plt.bar(index + bar_width, metrics_data['Recall (M)'], bar_width, label='Recall (M)', color='lightcoral')

plt.xlabel('Dataset')
plt.ylabel('Value')
plt.title('Mask Precision and Recall')
plt.xticks(index + bar_width / 2, datasets)
plt.legend()
plt.ylim(0, 1.0)

plt.tight_layout()
output_dir = "evaluation_results"
os.makedirs(output_dir, exist_ok=True)
plt.savefig(os.path.join(output_dir, "prec_rec.png"))
plt.show()

In [ ]:
# Plot mAP metrics
plt.figure(figsize=(16, 8))

# mAP for bounding boxes
plt.subplot(1, 2, 1)
plt.bar(index, metrics_data['mAP50 (B)'], bar_width, label='mAP50 (B)', color='teal')
plt.bar(index + bar_width, metrics_data['mAP50-95 (B)'], bar_width, label='mAP50-95 (B)', color='darkturquoise')

plt.xlabel('Dataset')
plt.ylabel('mAP')
plt.title('Bounding Box mAP')
plt.xticks(index + bar_width / 2, datasets)
plt.legend()
plt.ylim(0, 1.0)

# mAP for masks
plt.subplot(1, 2, 2)
plt.bar(index, metrics_data['mAP50 (M)'], bar_width, label='mAP50 (M)', color='teal')
plt.bar(index + bar_width, metrics_data['mAP50-95 (M)'], bar_width, label='mAP50-95 (M)', color='darkturquoise')

plt.xlabel('Dataset')
plt.ylabel('mAP')
plt.title('Mask mAP')
plt.xticks(index + bar_width / 2, datasets)
plt.legend()
plt.ylim(0, 1.0)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "map.png"))
plt.show()

In [ ]:
# Plot speed metrics
plt.figure(figsize=(14, 6))
plt.bar(index, metrics_data['Inference Time (ms)'], bar_width, label='Inference Time', color='mediumpurple')
# plt.bar(index + bar_width, metrics_data['NMS Time (ms)'], bar_width, label='NMS Time', color='mediumorchid')

plt.xlabel('Dataset')
plt.ylabel('Time (ms)')
plt.title('Processing Speed')
plt.xticks(index + bar_width / 2, datasets)
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "speed.png"))
plt.show()

## 5. Comparative Metrics Heatmap

A heatmap provides a clear visualization of how metrics compare across datasets.

In [ ]:
# Create a heatmap of metrics
performance_metrics = ['Precision (B)', 'Recall (B)', 'mAP50 (B)', 'mAP50-95 (B)', 
                       'Precision (M)', 'Recall (M)', 'mAP50 (M)', 'mAP50-95 (M)']

# Extract just the performance metrics (not speed)
heatmap_df = metrics_df[['Dataset'] + performance_metrics].set_index('Dataset')

# Create heatmap
plt.figure(figsize=(16, 8))
sns.heatmap(heatmap_df, annot=True, cmap='YlGnBu', fmt='.3f', linewidths=.5, vmin=0, vmax=1)
plt.title('Performance Metrics Across Datasets')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "heatmap.png"))
plt.show()

## 6. Sample Predictions Visualization

Let's visualize some sample predictions from each dataset to qualitatively assess model performance.

In [ ]:
import time

In [ ]:
def visualize_predictions(model, image_path, output_path=None, show=True):
    """Run prediction on an image and visualize results"""
    # Run prediction
    results = model.predict(image_path, save=False, verbose=False)
    result = results[0]  # Get first result
    
    # Get the image with annotations
    annotated_img = result.plot()
    
    # Convert from BGR to RGB for display
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    
    # Save if output path is provided
    if output_path:
        cv2.imwrite(output_path, annotated_img)
    
    # Display
    if show:
        plt.figure(figsize=(12, 8))
        plt.imshow(annotated_img_rgb)
        plt.axis('off')
        plt.title(f"Predictions on {Path(image_path).name}")
        plt.show()
    
    return result

def show_dataset_samples(model, dataset_path, num_samples=3, dataset_name=""):
    """Visualize predictions on random samples from a dataset"""
    print(f"\n{dataset_name} Dataset Sample Predictions:")
    
    # Get all image files
    image_files = list(Path(dataset_path).glob('*.jpg')) + list(Path(dataset_path).glob('*.png'))
    
    if not image_files:
        print(f"No images found in {dataset_path}")
        return
    
    # Select random samples
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))
    
    # Visualize each sample
    for img_path in sample_images:
        print(f"Predicting on {img_path.name}")
        # Create unique output filename
        output_filename = f"{dataset_name.lower()}_sample_{img_path.stem}_{int(time.time())}.jpg"
        output_filepath = os.path.join(output_dir, output_filename)
        
        # Run prediction and save visualization
        result = visualize_predictions(model, str(img_path), output_path=output_filepath)

In [ ]:
# Visualize predictions on training set
# Visualize predictions on training set
train_images_path = os.path.join(data_dir, 'train/images')
show_dataset_samples(model, train_images_path, num_samples=3, dataset_name="Training")

In [ ]:
# Visualize predictions on validation set
val_images_path = os.path.join(data_dir, 'val/images')
show_dataset_samples(model, val_images_path, num_samples=3, dataset_name="Validation")

## 7. Confusion Matrix Analysis

Let's analyze the confusion matrix to understand which classes are being confused with each other.

In [ ]:
# Get class names from the data configuration
class_names = data_config.get('names', [])
num_classes = len(class_names)

print(f"Model predicts {num_classes} classes: {class_names}")

# Plot confusion matrices if available
train_images_path = os.path.join(data_dir, 'train/images')
val_images_path = os.path.join(data_dir, 'val/images')

for dataset_path, metrics, name in zip(
    [train_images_path, val_images_path], 
    [train_metrics, val_metrics], 
    ['Training', 'Validation']
):
    if hasattr(metrics, 'confusion_matrix') and metrics.confusion_matrix is not None:
        conf_matrix = metrics.confusion_matrix
        
        plt.figure(figsize=(12, 10))
        sns.heatmap(
            conf_matrix.matrix / conf_matrix.matrix.sum(0), 
            annot=True, 
            fmt='.2f', 
            square=True,
            cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names
        )
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix - {name} Dataset')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"confusion_matrix_{name.lower()}.png"))
        plt.show()
    else:
        print(f"Confusion matrix not available for {name} dataset")

## 10. Summary and Conclusions

Based on our comprehensive evaluation, we can draw the following conclusions about our model:

1. **Overall Performance**: The model achieved a mask mAP50-95 of approximately 0.117 on the test set, which represents its ability to accurately detect and segment food items.

2. **Dataset Comparison**: 
   - The model performs best on the training set, as expected
   - Performance on validation and test sets is similar, suggesting good generalization
   - The gap between training and test performance indicates some overfitting, but it's within reasonable limits

3. **Strengths and Weaknesses**:
   - The model is efficient with fast inference times
   - Precision is generally higher than recall, meaning the model is more conservative in its predictions
   - Segmentation performance is slightly lower than detection performance

4. **Recommendations for Improvement**:
   - Collect more diverse training data
   - Try data augmentation techniques to improve generalization
   - Experiment with longer training or different learning rate schedules
   - Consider model ensemble approaches for critical applications

The final model provides a solid foundation for food recognition tasks and can be deployed for refrigerator content analysis.